In [1]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import datetime
import re
import os

In [2]:
df_total = pd.read_csv('ib_diploma_totals2022.csv')

In [3]:
df_total.rename(columns = {'LASID': 'lasid'}, inplace = True)

In [4]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   lasid                       132 non-null    int64 
 1   EE/TOK points:              132 non-null    int64 
 2   Total points:               132 non-null    int64 
 3   Result:                     132 non-null    object
 4   Diploma requirements code:  20 non-null     object
dtypes: int64(3), object(2)
memory usage: 5.3+ KB


In [5]:
df_total

,lasid,EE/TOK points:,Total points:,Result:,Diploma requirements code:
0,20136600,0,34,Diploma awarded,NaN
1,20137080,0,32,Diploma awarded,NaN
2,20134840,0,32,Diploma not awarded,Guilty of academic misconduct
3,20187517,0,32,Diploma awarded,NaN
4,20181104,0,32,Diploma awarded,NaN
...,...,...,...,...,...
127,20228493,1,32,Diploma awarded,NaN
128,20225229,1,33,Diploma awarded,NaN
129,20226583,1,31,Diploma awarded,NaN
130,20221493,1,31,Diploma awarded,NaN


In [6]:
df_component= pd.read_csv('ib_component_scores2022.csv')
df_component

,LASID,Graduation Year,IBAssessmentID,Raw mark,Moderated mark,Scaled mark,Component grade
0,20120010,2012,ENGLHL1,NaN,11.0,11.000,4
1,20120010,2012,ENGLHL2,NaN,20.0,20.000,7
2,20120010,2012,ENGLHLIA,21.0,19.0,19.000,5
3,20120010,2012,ENGLHLWRI,NaN,26.0,13.000,5
4,20120010,2012,HISTHL1,NaN,13.0,10.400,5
...,...,...,...,...,...,...,...
3024,20238695,2022,CHEMSLPRA,NaN,7.0,5.833,3
3025,20236101,2022,CHEMSLPRA,NaN,7.0,5.833,3
3026,20238867,2022,CHEMSLPRA,NaN,11.0,9.167,4
3027,20238824,2022,CHEMSLPRA,NaN,6.0,5.000,2


In [7]:
df_component.columns = ['lasid', 'grad_year', 'assessmentID', 'raw_mark',
       'moderated_mark', 'scaled_mark', 'component_grade']

In [8]:
df_component.component_grade.replace(['B', 'A', 'C', 'N', 'D', 'E'], 0, inplace = True)
df_component.component_grade = df_component.component_grade.astype('float64')
df_component.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3029 entries, 0 to 3028
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lasid            3029 non-null   int64  
 1   grad_year        3029 non-null   int64  
 2   assessmentID     3029 non-null   object 
 3   raw_mark         748 non-null    float64
 4   moderated_mark   2994 non-null   float64
 5   scaled_mark      2996 non-null   float64
 6   component_grade  3024 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 165.8+ KB


In [9]:
'''Add subject column to df_components'''
conditions = [
    (df_component['assessmentID'].str[0:4] == "BIOL") | (df_component['assessmentID'].str[0:4] == "CHEM") |\
    (df_component['assessmentID'].str[0:4] == "PHYS"),
    (df_component['assessmentID'].str[0:4] == "ENGL"),
    (df_component['assessmentID'].str[0:4] == "FREN") | (df_component['assessmentID'].str[0:4] == "SPAN") | (df_component['assessmentID'].str[0:4] == "GERM"),
    (df_component['assessmentID'].str[0:4] == "HIST"),
    (df_component['assessmentID'].str[0:4] == "MATH"),
    (df_component['assessmentID'].str[0:4] == "MUSI") | (df_component['assessmentID'].str[0:4] == "THEA") | (df_component['assessmentID'].str[0:4] == "VISU"),
    (df_component['assessmentID'].str[0:4] == "PSYC")
]

choices = [
    'Science', 'Literature', 'World Language', 'History', 'Math', 'Arts', 'Psychology'
]

df_component['subject'] = np.select(conditions, choices)
df_component

,lasid,grad_year,assessmentID,raw_mark,moderated_mark,scaled_mark,component_grade,subject
0,20120010,2012,ENGLHL1,NaN,11.0,11.000,4.0,Literature
1,20120010,2012,ENGLHL2,NaN,20.0,20.000,7.0,Literature
2,20120010,2012,ENGLHLIA,21.0,19.0,19.000,5.0,Literature
3,20120010,2012,ENGLHLWRI,NaN,26.0,13.000,5.0,Literature
4,20120010,2012,HISTHL1,NaN,13.0,10.400,5.0,History
...,...,...,...,...,...,...,...,...
3024,20238695,2022,CHEMSLPRA,NaN,7.0,5.833,3.0,Science
3025,20236101,2022,CHEMSLPRA,NaN,7.0,5.833,3.0,Science
3026,20238867,2022,CHEMSLPRA,NaN,11.0,9.167,4.0,Science
3027,20238824,2022,CHEMSLPRA,NaN,6.0,5.000,2.0,Science


In [10]:
df_component.loc[df_component['assessmentID'] == "CHEMISTRY", 'assessmentID'] = "CHEMSLMCQ"
df_component.loc[df_component['assessmentID'] == "MATHEMATICS", 'assessmentID'] = "MATHSL1"
df_component.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3029 entries, 0 to 3028
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lasid            3029 non-null   int64  
 1   grad_year        3029 non-null   int64  
 2   assessmentID     3029 non-null   object 
 3   raw_mark         748 non-null    float64
 4   moderated_mark   2994 non-null   float64
 5   scaled_mark      2996 non-null   float64
 6   component_grade  3024 non-null   float64
 7   subject          3029 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 189.4+ KB


In [11]:
print(df_component.loc[df_component['assessmentID'] == "MATHEMATICS"])

Empty DataFrame
Columns: [lasid, grad_year, assessmentID, raw_mark, moderated_mark, scaled_mark, component_grade, subject]
Index: []


In [12]:
df_component = df_component.merge(df_total[['lasid', 'Total points:']], how='left', on = 'lasid').sort_values('lasid')
df_component_groupby = df_component.groupby(['subject', 'assessmentID', 'Total points:'])[['component_grade']].mean()
df_component_groupby.drop('0', level = 0, axis = 0, inplace=True)
df_component_groupby.reset_index(inplace=True)
df_component_groupby

,subject,assessmentID,Total points:,component_grade
0,Arts,MUSIHL1,25.0,2.0
1,Arts,MUSIHLCRE,25.0,5.0
2,Arts,MUSIHLMLI,25.0,2.0
3,Arts,MUSIHLTHTR,25.0,4.0
4,Arts,MUSISL1,32.0,5.0
...,...,...,...,...
790,World Language,SPANSLWRI,33.0,4.5
791,World Language,SPANSLWRI,34.0,4.5
792,World Language,SPANSLWRI,35.0,6.5
793,World Language,SPANSLWRI,36.0,6.0


In [13]:
df_subject = pd.read_csv('ib_subject_scores2022.csv')
df_subject

,LASID,IBSubjectID,Category,Predicted grade,Grade,Scaled Total
0,20208391,CHEMSL,ANTICIPATED,2,2,17.0
1,20206407,CHEMSL,ANTICIPATED,3,4,43.0
2,20206227,CHEMSL,ANTICIPATED,3,3,33.0
3,20205333,CHEMSL,ANTICIPATED,3,3,39.0
4,20201551,CHEMSL,ANTICIPATED,3,2,27.0
...,...,...,...,...,...,...
850,20225229,SPANSL,DIPLOMA,4,6,69.0
851,20228493,HISTHL,DIPLOMA,4,6,70.0
852,20221490,HISTHL,DIPLOMA,5,6,70.0
853,20221490,ENGLHL,DIPLOMA,5,6,71.0


In [14]:
'''Add subject column to df_subject'''
conditions = [
    (df_subject['IBSubjectID'].str[0:4] == "BIOL") | (df_subject['IBSubjectID'].str[0:4] == "CHEM") |\
    (df_subject['IBSubjectID'].str[0:4] == "PHYS"),
    (df_subject['IBSubjectID'].str[0:4] == "ENGL"),
    (df_subject['IBSubjectID'].str[0:4] == "FREN") | (df_subject['IBSubjectID'].str[0:4] == "SPAN") | (df_subject['IBSubjectID'].str[0:4] == "GERM"),
    (df_subject['IBSubjectID'].str[0:4] == "HIST"),
    (df_subject['IBSubjectID'].str[0:4] == "MATH"),
    (df_subject['IBSubjectID'].str[0:4] == "MUSI") | (df_subject['IBSubjectID'].str[0:4] == "THEA") | (df_subject['IBSubjectID'].str[0:4] == "VISU"),
    (df_subject['IBSubjectID'].str[0:4] == "PSYC")
]

choices = [
    'Science', 'Literature', 'World Language', 'History', 'Math', 'Arts', 'Psychology'
]

df_subject['subject'] = np.select(conditions, choices)
df_subject

,LASID,IBSubjectID,Category,Predicted grade,Grade,Scaled Total,subject
0,20208391,CHEMSL,ANTICIPATED,2,2,17.0,Science
1,20206407,CHEMSL,ANTICIPATED,3,4,43.0,Science
2,20206227,CHEMSL,ANTICIPATED,3,3,33.0,Science
3,20205333,CHEMSL,ANTICIPATED,3,3,39.0,Science
4,20201551,CHEMSL,ANTICIPATED,3,2,27.0,Science
...,...,...,...,...,...,...,...
850,20225229,SPANSL,DIPLOMA,4,6,69.0,World Language
851,20228493,HISTHL,DIPLOMA,4,6,70.0,History
852,20221490,HISTHL,DIPLOMA,5,6,70.0,History
853,20221490,ENGLHL,DIPLOMA,5,6,71.0,Literature


In [15]:
df_subject = df_subject.merge(df_total[['lasid', 'Total points:']], how = "left", left_on = 'LASID', right_on = 'lasid' )
df_subject

,LASID,IBSubjectID,Category,Predicted grade,Grade,Scaled Total,subject,lasid,Total points:
0,20208391,CHEMSL,ANTICIPATED,2,2,17.0,Science,20208391.0,21.0
1,20206407,CHEMSL,ANTICIPATED,3,4,43.0,Science,20206407.0,25.0
2,20206227,CHEMSL,ANTICIPATED,3,3,33.0,Science,20206227.0,25.0
3,20205333,CHEMSL,ANTICIPATED,3,3,39.0,Science,20205333.0,33.0
4,20201551,CHEMSL,ANTICIPATED,3,2,27.0,Science,20201551.0,27.0
...,...,...,...,...,...,...,...,...,...
850,20225229,SPANSL,DIPLOMA,4,6,69.0,World Language,20225229.0,33.0
851,20228493,HISTHL,DIPLOMA,4,6,70.0,History,20228493.0,32.0
852,20221490,HISTHL,DIPLOMA,5,6,70.0,History,20221490.0,33.0
853,20221490,ENGLHL,DIPLOMA,5,6,71.0,Literature,20221490.0,33.0


In [16]:
df_subject_summary = df_subject.groupby(['subject', 'Total points:'])[['Grade', 'Scaled Total']].mean().reset_index()
df_subject_summary.drop(0, axis = 0, inplace = True)
df_subject_summary

,subject,Total points:,Grade,Scaled Total
1,Arts,13.0,0.000000,NaN
2,Arts,20.0,4.000000,45.000000
3,Arts,21.0,3.000000,33.000000
4,Arts,23.0,4.000000,39.000000
5,Arts,24.0,4.500000,49.000000
...,...,...,...,...
147,World Language,35.0,5.571429,67.000000
148,World Language,36.0,7.000000,91.000000
149,World Language,37.0,6.333333,80.666667
150,World Language,39.0,6.000000,84.000000


In [17]:
'''Create starting graph'''
figure = px.histogram(df_total, x="Total points:", nbins = 40, title = "Total IB Points Earned")
figure.update_layout(title_x = .5, xaxis_range=[18,42],  margin=dict(
        l=40,
        r=40,
        b=40,
        t=40,
        pad=2), 
        title_font_color = "blue", title_font_family = 'serif', title_font_size = 24,
        xaxis_title = "Total Points",
        yaxis_title = "Count")


'''Instantiate DASH app'''
app = JupyterDash(__name__,
               meta_tags = [{'name': 'viewport',
                            'content': 'width=device-width, intitial-scale = 1.0'}]
                             )

'''DASH app layout with Bootstrap'''
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Img(src="/assets/edulogo.jpg", style = {"width" : "75px", "height": "75px", 'padding': "10px"}),
            html.Span('IB DASHBoard',  style = {"font-family": "serif", "font-size": "32px"})
        ], width = 12, class_name = 'header_box'),
    ], justify = "center"),

    dbc.Row([
        dbc.Col([
            html.Label('1. Hover over a bar to select a point total:', style = {"color": "black", "font-size": "20px"}),
            dcc.Graph(
            id='overall_score', style={'width': '100vh', 'height': '75vh', 'padding':'40px'}, hoverData=None,
            figure = figure
            ),
            html.P()
        ], width = 6, align = "center"),

        dbc.Col([
            html.Label('2. Hover over a bar to select a subject to get its component scores:', style = {"color": "black", "font-size": "20px"}),
            dcc.Graph(
            id='subject_scores', style={'width': '100vh', 'height': '40vh', 'padding':'10px'}, figure={}
            ),
            
            dcc.Graph(
            id='subject_breakdown', style={'width': '100vh', 'height': '40vh'}, figure={}
            ),
        
            dcc.Store(id='intermediate_value', storage_type = 'session')
        ], width = 6, align = 'center'),
    ]),
], fluid=True,  style={"height": "100vh", "background-color" : "#fff", 'font-family': 'Lucida Sans'})
    
'''app callbacks'''
@app.callback(
    Output(component_id = 'subject_scores', component_property = 'figure'),
    Output(component_id = 'intermediate_value', component_property = 'data'),
    Input(component_id = 'overall_score', component_property = 'hoverData')
)
def update_subjects(hover_data):
    if hover_data is None:
        dff = df_subject_summary.loc[df_subject_summary['Total points:'] == 32]
        fig = px.bar(dff, x='subject', y="Grade")
        fig.update_layout(title_text = "Average Subject Scores", title_x = .5,  margin=dict(
                    l=40,
                    r=40,
                    b=40,
                    t=40,
                    pad=2),
                    title_font_size = 20)
        return fig, dff.to_json()
    else:
        print(f"hover data: {hover_data}")
        hov_total = hover_data['points'][0]['x']
        dff = df_subject_summary.loc[df_subject_summary['Total points:'] == hov_total]
        fig = px.bar(dff, x='subject', y="Grade", text_auto='.2s')
        fig.update_layout(title_text = f"Average Subject Scores when Total IB Points = {hov_total}", title_x = .5, \
                          yaxis_range=[1,7],  margin=dict(
                                        l=40,
                                        r=40,
                                        b=40,
                                        t=40,
                                        pad=2), title_font_size = 20)
        fig.update_traces(textposition = "outside")
        return fig, dff.to_json()

@app.callback(
    Output(component_id = 'subject_breakdown', component_property = 'figure'),
    Input(component_id = 'subject_scores', component_property = 'hoverData'),
    Input(component_id = 'overall_score', component_property = 'hoverData')
)
def update_components(hover_subject, hover_overall):
    if hover_subject is None:
        dff = df_component_groupby.loc[(df_component_groupby['subject'] == 'History') & (df_component_groupby['Total points:'] == 35)]
        fig = px.bar(dff, x='assessmentID', y="component_grade")
        fig.update_layout(title_text = "Breakdown of History", title_x = .5,  margin=dict(
                        l=40,
                        r=40,
                        b=40,
                        t=40,
                        pad=2),
                        title_font_size = 20)
        return fig
    else:
        hov_subject = hover_subject['points'][0]['x']
        hov_points = hover_overall['points'][0]['x']
        dff = df_component_groupby.loc[(df_component_groupby['subject'] == hov_subject) & (df_component_groupby['Total points:'] == hov_points)]
        fig = px.bar(dff, x='assessmentID', y="component_grade", text_auto='.2s')
        fig.update_layout(title_text = f"Component breakdown of {hov_subject} for students with {hov_points} points", title_x = .5, yaxis_range=[0,7],\
                          margin=dict(
                                        l=40,
                                        r=40,
                                        b=40,
                                        t=40,
                                        pad=2),
                         title_font_size = 20)
        fig.update_traces(textposition = "outside")
        return fig

if __name__ == '__main__':
    app.run_server(debug = True, port = 3005)

Dash app running on http://127.0.0.1:3005/
